In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Import the libraries

In [2]:
import os
import numpy as np
import random
import cv2
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler


#Load and Preprocess the Training Data

In [5]:
train_path = '/content/drive/MyDrive/Prodigy_3/train'

In [6]:
import random
train_images = []
train_labels = []

for filename in os.listdir(train_path):
    img_path = os.path.join(train_path, filename)
    img = cv2.imread(img_path)
    img = cv2.resize(img, (64, 64))  # Resize images to 64x64
    img = img.flatten()


    if "cat" in filename:
        label = 1  # 1 for cats
    elif "dog" in filename:
        label = 0  # 0 for dogs

    train_images.append(img)
    train_labels.append(label)

train_images = np.array(train_images)
train_labels = np.array(train_labels)

combined = list(zip(train_images, train_labels))
random.shuffle(combined)

subset_size = int(0.5 * len(combined))
subset = combined[:subset_size]

train_images, train_labels = zip(*subset)
train_images = np.array(train_images)
train_labels = np.array(train_labels)


#Split the Training Data into Train and Validation Sets

In [7]:
# Split the subset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(train_images, train_labels, test_size=0.2, random_state=42)

#Feature Scaling

In [8]:
# Feature scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

#Train the SVM Model

In [9]:
svm = SVC(kernel='linear')
svm.fit(X_train, y_train)

SVC(kernel='linear')

#Validate the Model

In [10]:
y_pred = svm.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
print(f"Validation Accuracy: {accuracy:.4f}")
print(classification_report(y_val, y_pred, target_names=['Cat', 'Dog']))

Validation Accuracy: 0.5580
              precision    recall  f1-score   support

         Cat       0.56      0.53      0.54      1244
         Dog       0.56      0.59      0.57      1256

    accuracy                           0.56      2500
   macro avg       0.56      0.56      0.56      2500
weighted avg       0.56      0.56      0.56      2500



In [11]:
# Define path to the test dataset
test_path = '/content/drive/MyDrive/Prodigy_3/test1'

# Initialize list for test images
test_images = []
image_names = []

# Load and preprocess test images
for filename in os.listdir(test_path):
    img_path = os.path.join(test_path, filename)
    img = cv2.imread(img_path)
    img = cv2.resize(img, (64, 64))  # Resize images to 64x64

    # Flatten the image to convert to a 1D array
    img = img.flatten()

    test_images.append(img)
    image_names.append(filename)

test_images = np.array(test_images)


In [12]:
# Apply the same scaling to the test data
test_images = scaler.transform(test_images)

In [13]:
test_predictions = svm.predict(test_images)

In [14]:
import pandas as pd

submission_df = pd.DataFrame({
    'Filename': image_names,
    'Prediction': test_predictions
})
submission_df['Prediction'] = submission_df['Prediction'].map({0: 'Cat', 1: 'Dog'})
submission_path = '/content/drive/MyDrive/submission.csv'
submission_df.to_csv(submission_path, index=False)

print(f"Submission saved to {submission_path}")

Submission saved to /content/drive/MyDrive/submission.csv
